## FDW Crop Production Data Profiling - Lesotho

In [1]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('../data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32635' # Lesotho
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/LS_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Lesotho',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, auth=auth, params=parameters, proxies={})
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1980	 | LS1980A1 | 10	| nan	| 0	| nan	| 0	|
----------------------------------------------- #


In [3]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# 2. Manual setting
# a) Add admin names to reporting units
admin_level = 'admin_1'
reporting_unit = df.loc[df['fnid'].map(lambda x: x[6] != 'A'), 'fnid'].unique()
df.loc[df.fnid.isin(reporting_unit),admin_level] = df.loc[df.fnid.isin(reporting_unit), 'locality_name'].map(lambda x: x.split(' ')[0])
# 3. Remove duplicate data 
pub_name_remove = 'BOS, Lesotho, Statisitical Report, 2017/2018 Crop Forecasting Report, NO 21 of 2040'
df = df[df['publication_name'] != pub_name_remove]
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df, area_priority='Area Planted')
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([LS_Admin1_1980], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 3,519
Removed 136 "Missing Value" points
1,170/1,173 "Area Planted" points are retained.
1,108/1,173 "Quantity Produced" points are retained.
1,105/1,173 "Yield" points are retained.
Current data points: 3,383

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1981 - 2018
7 grain types are found: Barley (Unspecified), Maize (Corn), Maize Grain (White), Oats (Unspecified), Sorghum, Sorghum grain, Wheat Grain
2 seasons are found: Summer (08-01), Winter (02-01)
1 crop production system are found: none
Data sources include:
[1] Bureau of Statistics, Lesotho --- Agricultural Statistical Report, Lesotho
Administrative-1 fnids: 10
Administrative-2 fnids: 0
0 reporting units are found: 

- Total production over time ------------------ #
season_name         Summer Winter
                                 
Maize (Corn)          1.3%   0.0%
Maize Grain (White)  71

- Lesotho crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-lesotho.png?itok=rDGlnOwG)

![USDA](https://ipad.fas.usda.gov/countrysummary/images/LT/cropcalendar/safrica_lt_calendar.png)

- FDW data consists of `LS1980A1`.

| Year | Admin-1 | # units  | Admin-2  | # units |
| :---: | :----:  | :----:   | :----:   | :---:  |
| 1980	 | `LS1980A1` | 10	| -	| -	|

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/LS_admin_shapes.png?raw=true)

- **`LS1980A1`** is used to represent current admin-level 1 crop data.
- Lesotho has 2 crop seasons: `Summer (08-01)` and `Winter (02-01)`.
- Lesotho has no crop production system.
- Lesotho has no population group.

In [4]:
# Define the latest shapefile ------------------- #
latest_level = 1
shape_latest = LS_Admin1_1980.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #

In [5]:
# Link admin boundaries ------------------------- #
link, over = FDW_PD_CreateAdminLink(LS_Admin1_1980, LS_Admin1_1980, 'ADMIN1', 'ADMIN1', prod, epsg)
# Crop specific ratios
link_ratio = FDW_PD_RatioAdminLink(link, prod, over, mdx_pss)
# Add current unit to link_ratio
for fnid_new in link_ratio.keys():
    link_ratio[fnid_new][fnid_new] = 1.0
    link_ratio[fnid_new] = link_ratio[fnid_new].sort_index(axis=1, ascending=False)
# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #

In [6]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new
# ----------------------------------------------- #

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = crop_new.T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack_gscd = stack[names]
stack_gscd['gscd_code'] = 'calibrated'
# ----------------------------------------------- #

# Reported FDW data ----------------------------- #
stack_fdw = df[names]
stack_fdw['indicator'] = stack_fdw['indicator'].replace({'Area Harvested':'area','Quantity Produced':'production','Yield':'yield'})
stack_fdw['gscd_code'] = 'reported'
# ----------------------------------------------- #

# Final Processing ------------------------------ #
stack = pd.concat([stack_fdw,stack_gscd], axis=0).reset_index(drop=True)
# No concerns found for grain types
stack['product'] = stack['product'].replace(product_category)
# Calibration of planting and Harvest year and season
cs = {
    'Summer': {'planting_month':{'08-01':'10-01'},'harvest_month':{'01-01':'07-01'}},
    'Winter': {'planting_month':{'02-01':'05-01'},'harvest_month':{'07-01':'11-01'}},
}
cy = {'Summer': {'planting_year':-1, 'harvest_year':-1}}
stack, df, link_ratio = FDW_PD_CaliSeasonYear(stack, df, link_ratio, cs, cy)
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

# Save data
fn_out = '../data/crop/adm_crop_production_LS.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')
save_hdf('../data/crop/adm_crop_production_LS.hdf', stack)
save_hdf('../data/crop/adm_crop_production_LS_raw.hdf', df)
save_npz('../data/crop/adm_crop_production_LS_ratio.npz', link_ratio)

../data/crop/adm_crop_production_LS.csv is saved.
../data/crop/adm_crop_production_LS.hdf is saved.
../data/crop/adm_crop_production_LS_raw.hdf is saved.
../data/crop/adm_crop_production_LS_ratio.npz is saved.


## Visualization of production data

In [7]:
# Bar chart of national grain production
country_iso, country_name = 'LS', 'Lesotho'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Maize','Sorghum','Wheat']
for season_name in ['Summer','Winter']:
    footnote = 'National grain production in %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote, fn_save)
    fig.show()

../figures/LS_bar_natgrainprod_Summer.png is saved.


../figures/LS_bar_natgrainprod_Winter.png is saved.


In [8]:
# Lineplot of Production-Area-Yield (PAY) time-series
country_iso, country_name = 'LS', 'Lesotho'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Maize','Summer'],
    ['Sorghum','Summer'],
    ['Wheat','Summer'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield (PAY) time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote, fn_save)
    fig.show()

../figures/LS_line_pay_Maize_Summer.png is saved.


../figures/LS_line_pay_Sorghum_Summer.png is saved.


../figures/LS_line_pay_Wheat_Summer.png is saved.


In [9]:
# Heatmap of seasonal data availability
country_iso, country_name = 'LS', 'Lesotho'
df = pd.read_hdf('../data/crop/adm_crop_production_%s_raw.hdf' % country_iso)
df['year'] = df['harvest_year']
code = {'Summer':1,'Winter':10}
comb = {1:1,10:2,11:3,12:3,20:3,21:3,22:3}
comb_name = {1:'Summer',2:'Winter',3:'Summer + Winter'}
for product_name in ['Maize Grain (White)', 'Wheat Grain']:
    data = df[(df['product'] == product_name) & (df['season_name'].isin(code.keys()))]
    footnote = 'Seasonal data availability in %s - %s (uncorrected)' % (country_name, product_name)
    fn_save = '../figures/%s_heat_seasondata_%s.png' % (country_iso, product_name)
    fig = PlotHeatSeasonData(data, code, comb, comb_name, footnote, fn_save)
    fig.show()

../figures/LS_heat_seasondata_Maize Grain (White).png is saved.


../figures/LS_heat_seasondata_Wheat Grain.png is saved.


In [10]:
# Calibrated PAY time-series per FNID
from tools_graphic import PlotLineCropTS
country_iso, country_name = 'LS', 'Lesotho'
df = pd.read_hdf('../data/crop/adm_crop_production_%s.hdf' % country_iso)
df = df[df['gscd_code']=='calibrated']; df['year'] = df['harvest_year']
link_ratio = load_npz('../data/crop/adm_crop_production_%s_ratio.npz' % country_iso)
year_all = np.arange(df['year'].min(), df['year'].max()+1)
prod_season = [
    ['Maize', 'Summer'],
    ['Sorghum', 'Summer'],
    ['Wheat', 'Summer'],
]
for product_name, season_name in prod_season:
    sub = df[
        (df['product'] == product_name) &
        (df['season_name'] == season_name)
    ]
    for fnid in sub['fnid'].unique():
        sub_fps = sub[sub['fnid'] == fnid]
        fn_save = '../figures/crop_calibrated/%s_%s_%s_%s.png' % (country_iso, product_name, season_name, fnid)
        fig = PlotLineCropTS(sub_fps, fnid, product_name, season_name, link_ratio, year_all, fn_save)

../figures/crop_calibrated/LS_Maize_Summer_LS1980A101.png is saved.
../figures/crop_calibrated/LS_Maize_Summer_LS1980A102.png is saved.
../figures/crop_calibrated/LS_Maize_Summer_LS1980A103.png is saved.
../figures/crop_calibrated/LS_Maize_Summer_LS1980A104.png is saved.
../figures/crop_calibrated/LS_Maize_Summer_LS1980A105.png is saved.
../figures/crop_calibrated/LS_Maize_Summer_LS1980A106.png is saved.
../figures/crop_calibrated/LS_Maize_Summer_LS1980A107.png is saved.
../figures/crop_calibrated/LS_Maize_Summer_LS1980A108.png is saved.
../figures/crop_calibrated/LS_Maize_Summer_LS1980A109.png is saved.
../figures/crop_calibrated/LS_Maize_Summer_LS1980A110.png is saved.
../figures/crop_calibrated/LS_Sorghum_Summer_LS1980A101.png is saved.
../figures/crop_calibrated/LS_Sorghum_Summer_LS1980A102.png is saved.
../figures/crop_calibrated/LS_Sorghum_Summer_LS1980A103.png is saved.
../figures/crop_calibrated/LS_Sorghum_Summer_LS1980A104.png is saved.
../figures/crop_calibrated/LS_Sorghum_Su